<a href="https://colab.research.google.com/github/RahulArya15/Caption-Generator/blob/main/caption_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!mkdir ~/.kaggle/

In [3]:
!cp kaggle.json ~/.kaggle

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d hsankesara/flickr-image-dataset


100% 8.15G/8.16G [01:08<00:00, 180MB/s]
100% 8.16G/8.16G [01:08<00:00, 129MB/s]


In [ ]:
! unzip flickr-image-dataset.zip

In [94]:
import string
import numpy as np
import pandas as pd
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
# small library for seeing the progress of loops.
from tqdm.notebook import tqdm #this was changed
tqdm().pandas()

0it [00:00, ?it/s]

In [105]:
df = pd.read_csv('/content/flickr30k_images/results.csv', sep = '|')
#df = df.set_index(df.iloc[0].values)
#print(df.head())
#print(df.unique)
#new_header = df.nunique
#print(df.iloc[0])
#print(df.head())
#print(df.shape)
print(df.iloc[0][0])
print("\n")
print(type(df.iloc[0][2]))
#print(df.iloc([0][1]))
#df.head()
#df.head()
#df.describe()

1000092795.jpg


<class 'str'>


In [96]:
#loading a text file into memory
def load_doc(filename):
  df = pd.read_csv(filename, sep = '|')
  return df

In [103]:
#get all images with their captions
def all_img_captions(filename):
  df = load_doc(filename)
  #captions = file.split('\n')
  descriptions = {}
  for i in range(len(df)):
    img = df.iloc[i][0]
    caption = df.iloc[i][2]
    if img[:-2] not in descriptions:
      descriptions[img[:-2]] = [caption]
    else:
      descriptions[img[:-2]].append(caption)
  return descriptions


In [108]:
#lower casing, removinf puntutations and words containing numbers
def cleaning_text(captions):
  table = str.maketrans('','', string.punctuation)
  for img, caps in captions.items():
    for i, img_caption in enumerate(caps):
      str(img_caption).replace("-"," ")
      desc = str(img_caption).split()

      #convers to lowercase
      desc = [word.lower() for word in desc]
      #remove punctuatin from each token
      desc = [word.translate(table) for word in desc]
      #removing 's and a
      desc = [word for word in desc if(len(word) > 1)]
      #remove tokens with numbers in them
      desc = [word for word in desc if(word.isalpha())]
      #convert back to string
      img_caption = ' '.join(desc)
      captions[img][i] = img_caption
  return captions

In [115]:
#build vocabulary of all unique words
def text_vocabulary(descriptions):
  vocab = set()
  for key in descriptions.keys() :
    [vocab.update(d.split()) for d in descriptions[key]]
  return vocab

In [100]:
#all description in one file
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(key + '\t' + desc)
  data = "\n".join(lines)
  file = open(filename,"w")
  file.write(data)
  file.close()

In [101]:
dataset_text = "/content/captions.txt"
dataset_images = "/content/Images"

In [116]:
#we prepare our text data
filename ='/content/flickr30k_images/results.csv'
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)
#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving each description to file 
save_descriptions(clean_descriptions, "descriptions.txt")

Length of descriptions = 31783
Length of vocabulary =  19735


In [114]:
print(type(clean_descriptions))

<class 'dict'>
